In [5]:
import tensorflow as tf

In [6]:
tf.test.is_gpu_available()

False

In [4]:
import os

import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers, preprocessing, optimizers

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
main_dir = "./dataset"
# classification_dirs = [("no_tumor",), ("meningioma_tumor",), ("glioma_tumor",), ("pituitary_tumor",) ]
classification_dirs = [("no_tumor",), ("meningioma_tumor",) ]
classification_dirs = [("no_tumor",), ("glioma_tumor",) ]
classification_dirs = [("no_tumor",), ("pituitary_tumor",) ]
classification_dirs = [("glioma_tumor",), ("pituitary_tumor",) ]
classification_dirs = [("meningioma_tumor",), ("pituitary_tumor",) ]
classification_dirs = [("meningioma_tumor",), ("glioma_tumor",) ]

resolution = 64

def load_images(root_dir_name):
    x = []
    y = []
    
    for label, sub_dir_names in enumerate(classification_dirs):
        for sub_dir_name in sub_dir_names:
            print(f"loading {root_dir_name} {sub_dir_name}")
            sub_dir_path = os.path.join(main_dir, root_dir_name, sub_dir_name)
            for image_name in os.listdir(sub_dir_path):
                image_path = os.path.join(sub_dir_path, image_name)
                image = preprocessing.image.load_img(image_path, color_mode="grayscale", target_size=(resolution, resolution))
                x.append(preprocessing.image.img_to_array(image))
                y.append(label)
    
    x = np.array(x) / 255.0
    y = np.array(y)
    
    return x, y

x_train, y_train = load_images("Training")
x_test, y_test = load_images("Testing")

print("done loading!")
print("\nshapes:")
print(x_train.shape)
print(x_test.shape)

In [ ]:
c = 8

fig, subplots = plt.subplots(1, c)
fig.set_size_inches(25, 3)
for i in range(c):
    n = np.random.randint(0, len(x_test))
    num = y_test[n]
    word = "out" if num == 0 else ""
    
    subplots[i].imshow(x_test[n].reshape((resolution, resolution)), cmap="gray")
    subplots[i].set_title(f"brain with{word} tumor: {num}")
    subplots[i].axis("off")
plt.show()

In [ ]:
model = keras.Sequential([
    layers.Input((resolution, resolution, 1)),
    layers.Conv2D(32, (3, 3), activation="linear", padding="same"),
    layers.MaxPooling2D(2, 2), # 32 x 32
    layers.Conv2D(64, (3, 3), activation="linear", padding="same"),
    layers.MaxPooling2D(2, 2), # 16 x 16
    layers.Conv2D(128, (3, 3), activation="linear", padding="same"),
    layers.MaxPooling2D(2, 2), # 8 x 8
    layers.Conv2D(32, (3, 3), activation="linear", padding="same"),
    layers.MaxPooling2D(2, 2), # 4 x 4
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(512, activation="linear"), # 128 and 256 got 94% after 15 epochs
    layers.Dropout(0.5),
    layers.Dense(256, activation="linear"),
    layers.Dense(4, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizers.RMSprop(), metrics=["accuracy"])
model.summary()

In [ ]:
history = model.fit(x_train, y_train, batch_size=4, shuffle=True, epochs=15, validation_data=(x_test, y_test))

In [ ]:
y_test_results = model.predict([x_test])

c = 8

fig, subplots = plt.subplots(1, c)
fig.set_size_inches(25, 3)
for i in range(c):
    n = np.random.randint(0, len(x_test))
    guess = str(round(y_test_results[n][0], 2)).ljust(4, "0")
    actual = y_test[n]

    subplot = subplots[i]
    subplot.imshow(x_test[n].reshape((resolution, resolution)), cmap="gray")
    subplot.set_title(f"guess: {guess}, actual: {actual}")
    subplot.axis("off")
plt.show()

In [7]:
main_dir = "./dataset"
# classification_dirs = [("no_tumor",), ("meningioma_tumor",), ("glioma_tumor",), ("pituitary_tumor",) ]
comb = {
    'no_tumor_to_meningioma_tumor': [("no_tumor",), ("meningioma_tumor",) ],
    'no_tumor_to_glioma_tumor': [("no_tumor",), ("glioma_tumor",) ],
    'no_tumor_to_pituitary_tumor': [("no_tumor",), ("pituitary_tumor",) ],
    'glioma_tumor_to_pituitary_tumor':[("glioma_tumor",), ("pituitary_tumor",) ],
    'meningioma_tumor_to_pituitary_tumor': [("meningioma_tumor",), ("pituitary_tumor",) ],
    'meningioma_tumor_to_glioma_tumor':[("meningioma_tumor",), ("glioma_tumor",) ]

}

dataset = {
    
}

resolution = 64

def load_images(root_dir_name, classification_dirs):
    x = []
    y = []
    
    for label, sub_dir_names in enumerate(classification_dirs):
        for sub_dir_name in sub_dir_names:
            print(f"loading {root_dir_name} {sub_dir_name}")
            sub_dir_path = os.path.join(main_dir, root_dir_name, sub_dir_name)
            for image_name in os.listdir(sub_dir_path):
                image_path = os.path.join(sub_dir_path, image_name)
                image = preprocessing.image.load_img(image_path, color_mode="grayscale", target_size=(resolution, resolution))
                x.append(preprocessing.image.img_to_array(image))
                y.append(label)
    
    x = np.array(x) / 255.0
    y = np.array(y)
    
    return x, y

for key in comb:
    x_train, y_train = load_images("Training", comb[key])
    x_test, y_test = load_images("Testing", comb[key])
    
    dataset[key] = {
        'x_train': x_train,
        'y_train': y_train,
        'x_test': x_test,
        'y_test': y_test
    }

print("done loading!")

loading Training no_tumor
loading Training meningioma_tumor
loading Testing no_tumor
loading Testing meningioma_tumor
loading Training no_tumor
loading Training glioma_tumor
loading Testing no_tumor
loading Testing glioma_tumor
loading Training no_tumor
loading Training pituitary_tumor
loading Testing no_tumor
loading Testing pituitary_tumor
loading Training glioma_tumor
loading Training pituitary_tumor
loading Testing glioma_tumor
loading Testing pituitary_tumor
loading Training meningioma_tumor
loading Training pituitary_tumor
loading Testing meningioma_tumor
loading Testing pituitary_tumor
loading Training meningioma_tumor
loading Training glioma_tumor
loading Testing meningioma_tumor
loading Testing glioma_tumor
done loading!


In [8]:
def make_model():
    model = keras.Sequential([
        layers.Input((resolution, resolution, 1)),
        layers.Conv2D(32, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(2, 2), # 32 x 32
        layers.Conv2D(64, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(2, 2), # 16 x 16
        layers.Conv2D(128, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(2, 2), # 8 x 8
        layers.Conv2D(64, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(2, 2), # 4 x 4
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(512, activation="relu"), # 128 and 256 got 94% after 15 epochs
        layers.Dropout(0.5),
        layers.Dense(256, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(64, activation="relu"),
        layers.Dense(1, activation="sigmoid")
    ])

    model.compile(loss="binary_crossentropy", optimizer=optimizers.RMSprop(), metrics=["accuracy"])
    model.summary()
    
    return model

In [9]:
models = {}
historys = {}
for key in dataset:
    print("Key: {}".format(key))
    model = make_model()
    x_train = dataset[key]['x_train']
    x_test = dataset[key]['x_test']
    y_train = dataset[key]['y_train']
    y_test = dataset[key]['y_test']
    history = model.fit(x_train, y_train, batch_size=4, shuffle=True, epochs=50, validation_data=(x_test, y_test))
    
    models[key] = model
    historys[key] = history

Key: no_tumor_to_meningioma_tumor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_3 (Conv2D)     

1653/1653 [==============================] - 21s 13ms/sample - loss: 0.5263 - accuracy: 0.7374 - val_loss: 3.8946 - val_accuracy: 0.4885
Epoch 2/15
1653/1653 [==============================] - 18s 11ms/sample - loss: 0.3643 - accuracy: 0.8845 - val_loss: 6.3331 - val_accuracy: 0.5402
Epoch 3/15
1653/1653 [==============================] - 18s 11ms/sample - loss: 0.3781 - accuracy: 0.9214 - val_loss: 5.4863 - val_accuracy: 0.5517
Epoch 4/15
1653/1653 [==============================] - 18s 11ms/sample - loss: 0.2828 - accuracy: 0.9341 - val_loss: 8.5606 - val_accuracy: 0.5690
Epoch 5/15
1653/1653 [==============================] - 18s 11ms/sample - loss: 0.2035 - accuracy: 0.9534 - val_loss: 13.8015 - val_accuracy: 0.5747
Epoch 6/15
1653/1653 [==============================] - 18s 11ms/sample - loss: 0.2971 - accuracy: 0.9431 - val_loss: 37.1151 - val_accuracy: 0.5460
Epoch 7/15
1653/1653 [==============================] - 18s 11ms/sample - loss: 0.2220 - accuracy: 0.9589 - val_loss: 8.0

1648/1648 [==============================] - 20s 12ms/sample - loss: 0.6890 - accuracy: 0.5200 - val_loss: 1.6941 - val_accuracy: 0.5628
Epoch 2/15
1648/1648 [==============================] - 17s 11ms/sample - loss: 0.6169 - accuracy: 0.6687 - val_loss: 14.6886 - val_accuracy: 0.5349
Epoch 3/15
1648/1648 [==============================] - 17s 11ms/sample - loss: 0.5540 - accuracy: 0.7573 - val_loss: 1.5538 - val_accuracy: 0.6186
Epoch 4/15
1648/1648 [==============================] - 17s 10ms/sample - loss: 0.5169 - accuracy: 0.7822 - val_loss: 7.9087 - val_accuracy: 0.6093
Epoch 5/15
1648/1648 [==============================] - 10914s 7s/sample - loss: 0.4718 - accuracy: 0.8083 - val_loss: 5.2426 - val_accuracy: 0.6233
Epoch 6/15
1648/1648 [==============================] - 14s 8ms/sample - loss: 0.4678 - accuracy: 0.8325 - val_loss: 3.6578 - val_accuracy: 0.6558
Epoch 7/15
1648/1648 [==============================] - 14s 8ms/sample - loss: 0.4586 - accuracy: 0.8331 - val_loss: 8.498